# EN2a Session 1: Fitting Dampened Harmonic Oscillator Data
This notebook will guide you through the basic steps of analysing the data you obtained from your experiment with the RLC circuit. Before you start, make sure that you have some good data from the experiment and that you have saved your data as one or more .csv files.

Don't forget to explain your work in your lab journal! You don't have to print your Python code and glue it in, but the steps of your reasoning should be clear from your lab journal *alone*. As always, a picture says more than a thousand words: putting your graphs in the lab journal is recommended.

By now, you have already finished the Programming Methods course, which means that you've already seen the NumPy and Matplotlib modules. We don't expect you to be fluent Python programmers yet, so we will walk you through the steps of fitting the data for this experiment. Later, you will be able to use these skills during the open experiments.

### Our goal
![Image of Yaktocat](https://www.seas.upenn.edu/~ese206/RLCresponse/Image3.gif)

Using the data you obtained, we want to do two things. First of all, we want to find the characteristic time $\tau$ of the pink exponential function (see figure). Secondly, we want to determine the frequency of the wave.

Our "plan of attack" will be: first, we filter only local maxima from your data. The exponential is meant to pass right through these maxima, so we will fit an exponential function through these maxima. From the parameters of the fit, we can read off the characteristic time.

We will need quite some tools to make this work. Obviously, we will be using NumPy and Matplotlib, but we will also use SciPy. SciPy is a library with specialized tools for scientific research. It contains, amongst others, functions for fitting functions to data, and finding local maxima.

### NumPy and Matplotlib Recap
To get reaquinted will NumPy and Matplotlib, study the next two cells of code. In the code, I define two NumPy arrays: one for the x- and one for the y-axis of my graph. The NumPy array for the y-axis is filled with a sine wave. I then graph the sine wave.

Try changing the frequency of the sine wave. Can you add labels and a title to the plot? And can you change the range of the x-axis? 

At the very least, try changing the program to use np.arange instead of np.linspace.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Plot in the notebook:
%matplotlib inline 

In [ ]:
x_axis = np.linspace(0, 10, 1000)
y_axis = np.sin(x_axis)

plt.plot(x_axis, y_axis)
plt.show()

### Finding maxima
Now we will try to write code that finds the local maxima of this function. We could do this using a for loop, but it is much simpler to use a function that has already been made. This function is called from argrelextrema and it is part of the scipy.signal module.





In [ ]:
# Always import modules and functions before using them!
from scipy.signal import argrelextrema

The following code finds all the maxima of the array and puts the indices the maxima into an array I've called "maxima". Try printing it to verify it works: how many elements do you expect in the array?

In [ ]:
maxima = argrelextrema(y_axis, np.greater)

In practise, you won't be looking for the indices of the maxima, but for the x- and y-coordinates of the maxima. The following cell demonstrates how you can go from an array of indices to an array of corresponding x- and y-coordinates. Use the plt.plot and plt.scatter functions to verify it worked, by plotting the sine wave and the maxima you found.

In [ ]:
x_max = x_axis[maxima]
y_max = y_axis[maxima]

### The maxima of your data
Now we will apply what we just learned to your data. Like with EN1a, I've already written most of the code, but you will have to fill in the blanks.

You will see that just writing argrelextrema(ydata, np.greater) will not work anymore, because the function finds local maxima. If there's some noise in your data, then there will be a lot of points that are a local maximum, albeit *very* locally.

To fix this, you will have to use argrelextrema's order parameter. This argument represents the range in which the maximum has to be a maximum. For example, if the order argument is equal to 100, then only maxima that are the largest within a 100-sample range will be accepted as local maxima.

Use Matplotlib to check your work!

In [ ]:
# First load your data: obviously you must fill in the path yourself
tdata_raw, vdata_raw = np.loadtxt("test.csv", unpack=True)

# Your data might not be perfectly cropped. Maybe the first few samples are not part of the measurement yet.
# Use a slice to specify the range over which we will be looking for maxima:
t, V = tdata_raw[:300], vdata_raw[:300]

# Now find the maxima:
maxima = argrelextrema(V, np.greater_equal, order=1)

print(maxima)

# Convert to coordinates:
tmax, Vmax = t[maxima], V[maxima]

# Plot:
plt.plot(t, V, color="blue")
plt.scatter(tmax, Vmax, color="red")
plt.ylim(-2,5)
#plt.xlim(0,0.00004)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (V)")

### Fitting an exponential curve through your maxima
Now we will use another function from SciPy called curve_fit to fit an exponential function through your data. You've already seen curve_fit in EN1a.

Recall: curve_fit takes at least three parameters. First, it takes the name of the function you want to fit. **You must have defined this function yourself, and its first argument should be the x-axis.** Second, it takes the x-components of your data, third it takes the y-components of your data.

curve_fit returns two arrays. It will return an array containing the optimal parameters, and a matrix containing the covariances on the parameters. You don't have to know what a covariance matrix represents yet, but it's good to know that the diagonal contains the variances (standard deviation squared) of the parameters.

In [ ]:
"""This code tries to fit a linear function to your data. 
   Your data isn't linear yet, but in EN1a you learned how to do this. It's smart to use np.log.
   Don't forget to check whether your data is exponential by plotting a log-plot.
   Also, make sure to include error bars in the plots that end up in your lab journal. Use plt.errorbars instead of
   plt.scatter. You can look up how plt.errorbars work in the documentation, but it is often faster to type plt.errorbars
   and press shift+tab.
   Lastly, if you run into any trouble with the functions, don't forget to look at the documentation!"""
from scipy.optimize import curve_fit

def linear_function(x, a, b):
    return a*x + b

param, pcov = curve_fit(linear_function, tmax, np.log(Vmax),sigma=)

plt.scatter(tmax, np.log(Vmax), color="red")
plt.plot(t, linear_function(t, param[0], param[1]))   

### Finding the frequency of your oscillator
Lastly, we want to find the frequency of the oscillator. Luckily, we've already done most of the heavy lifting.

Think about how we can determine the frequency using the data we've already acquired. Two hints to help you on your way:

- np.diff() takes an array as an argument and returns the difference between the elements. E.g. np.diff([1, 2, 3, 9]) = [1, 1, 6]

- np.mean() takes an array and returns the mean. E.g. np.mean([1, 2, 3]) = 2.0

